In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [3]:
tool.name

'wikipedia'

In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://python.langchain.com/docs/langserve/#using-add_routes")
documents = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(documents)
vectordb = Chroma.from_documents(documents, OpenAIEmbeddings())
retriver = vectordb.as_retriever()

In [9]:
vectordb.similarity_search_with_score("LangServe helps developers deploy LangChain runnables and chains as a REST API.")

[(Document(metadata={'description': 'Release Notes', 'language': 'en', 'source': 'https://python.langchain.com/docs/langserve/#using-add_routes', 'title': '🦜️🏓 LangServe | 🦜️🔗 LangChain'}, page_content='🦜️🏓 LangServe | 🦜️🔗 LangChain\n\n\n\n\n\n\n\n\nSkip to main contentThese docs will be deprecated and no longer maintained with the release of LangChain v1.0 in October 2025. Visit the v1.0 alpha docsIntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchOn this page🦜️🏓 LangServe\n\n\n\n\n\n[!WARNING]\nWe recommend using LangGraph Platform rather than LangServe for new projects.\nPlease see the LangGraph Platform Migration Guide for more information.\nWe will continue to accept bug fixes for LangServe from the community; however, we\nwill not be accepting new feature contributions.\n\nOverview\u200b\nLangServe helps developers\ndeploy LangChain runnables and chains\nas a REST API.\nThis library is integrated with

In [10]:
from langchain.tools.retriever import create_retriever_tool
retriver_tool = create_retriever_tool(
    retriver,
    ## Tool name
    "langserver_search",
    ## Tool Instructions for the LLM to know what to do with this tool
    "Search about information about LangServe. For any question related to LangServe."
)

In [11]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [12]:
tools = [tool, retriver_tool, arxiv]

In [13]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/diegoagd10/Projects/langchain/venv/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langserver_search', description='Search about information about LangServe. For any question related to LangServe.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x30c404160>, retriever=VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x1792fbb50>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x30c404670>, retriever=VectorSto

In [14]:
import os
from dotenv import load_dotenv

load_dotenv() # Load all environment variables
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['LANGSMITH_API_KEY' ] = os.getenv('LANGSMITH_API_KEY')
os.environ['LANGSMITH_TRACING'] = "true"
os.environ['LANGSMITH_PROJECT'] = os.getenv('LANGSMITH_PROJECT')
os.environ['LANGSMITH_WORKSPACE_ID'] = os.getenv("LANGSMITH_WORKSPACE_ID")

In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [15]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [17]:
### Agents
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)

In [ ]:
## Agent Executor
from langchain.agents import AgentExecutor
## Verbose shows you the process to resolve user query
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [19]:
agent_executor.invoke({"input": "Que es LangServe?"})



> Entering new AgentExecutor chain...

Invoking: `langserver_search` with `{'query': 'LangServe'}`


🦜️🏓 LangServe | 🦜️🔗 LangChain








Skip to main contentThese docs will be deprecated and no longer maintained with the release of LangChain v1.0 in October 2025. Visit the v1.0 alpha docsIntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchOn this page🦜️🏓 LangServe





[!WARNING]
We recommend using LangGraph Platform rather than LangServe for new projects.
Please see the LangGraph Platform Migration Guide for more information.
We will continue to accept bug fixes for LangServe from the community; however, we
will not be accepting new feature contributions.

Overview​
LangServe helps developers
deploy LangChain runnables and chains
as a REST API.
This library is integrated with FastAPI and
uses pydantic for data validation.
In addition, it provides a client that can be used to call into runnables deployed

{'input': 'Que es LangServe?',
 'output': 'LangServe es una herramienta que ayuda a los desarrolladores a desplegar elementos ejecutables y cadenas de LangChain como una API REST. Está integrada con FastAPI y utiliza pydantic para la validación de datos. Además, proporciona un cliente que se puede usar para llamar a elementos ejecutables desplegados en un servidor, y también se encuentra disponible un cliente JavaScript en LangChain.js.\n\nLangServe está diseñado principalmente para desplegar elementos ejecutables simples y trabajar con primitivas conocidas en langchain-core. Sin embargo, se recomienda el uso de la plataforma LangGraph en lugar de LangServe para nuevos proyectos, especialmente para aplicaciones más complejas, ya que LangGraph Cloud es más adecuada para estos propósitos.\n\nEs importante tener en cuenta ciertas limitaciones y advertencias de seguridad que han sido documentadas, así como pasos de instalación y ejemplos para facilitar su uso.'}

In [20]:
agent_executor.invoke({"input": "Cuentame hacerca de Machine Learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine Learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genEl aprendizaje automático (Machine Learning, ML) es un campo de estudio dentro de la inteligencia artificial que se centra en el desarrollo y estudio de algoritmos estadísticos capaces de aprender a partir de datos y generar predicciones o decisiones. Este área combina conceptos de la informática, estadísticas y matemáticas para construir modelos que pueden "aprender" de manera automática sin ser programados explícitamente para realizar tareas específicas.

> Finished chain.


{'input': 'Cuentame hacerca de Machine Learning',
 'output': 'El aprendizaje automático (Machine Learning, ML) es un campo de estudio dentro de la inteligencia artificial que se centra en el desarrollo y estudio de algoritmos estadísticos capaces de aprender a partir de datos y generar predicciones o decisiones. Este área combina conceptos de la informática, estadísticas y matemáticas para construir modelos que pueden "aprender" de manera automática sin ser programados explícitamente para realizar tareas específicas.'}

In [21]:
agent_executor.invoke({"input": "De que se trata el paper 1605.08386?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloEl paper titulado "Heat-bath random walks with Markov bases", publicado el 26 de mayo de 2016 por Caprice Stanley y Tobias Windisch, estudia grafos en puntos de una red cuyos bordes provienen de un conjunto finito de elementos.

> Finished chain.


{'input': 'De que se trata el paper 1605.08386?',
 'output': 'El paper titulado "Heat-bath random walks with Markov bases", publicado el 26 de mayo de 2016 por Caprice Stanley y Tobias Windisch, estudia grafos en puntos de una red cuyos bordes provienen de un conjunto finito de elementos.'}